In [1]:
@file:DependsOn("com.google.adk:google-adk:0.3.0")
@file:DependsOn("com.google.adk:google-adk-dev:0.2.0")
@file:DependsOn("com.google.genai:google-genai:1.23.0")
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-core:1.8.1")
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-rx3:1.8.1")
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-coroutines-reactive:1.8.1")

In [11]:
import com.google.adk.agents.LlmAgent

import com.google.adk.models.Gemini
import com.google.genai.Client

import com.google.adk.runner.InMemoryRunner
import com.google.adk.sessions.GetSessionConfig
import com.google.adk.sessions.Session
import com.google.adk.sessions.InMemorySessionService
import com.google.adk.tools.BaseTool
import com.google.adk.tools.mcp.McpTool
import com.google.adk.tools.mcp.McpToolset
import com.google.adk.tools.ToolContext
import com.google.adk.tools.mcp.McpSessionManager
import com.google.adk.tools.mcp.StdioServerParameters
import com.google.adk.tools.mcp.SseServerParameters
import com.google.adk.agents.ReadonlyContext

import com.google.genai.types.Content
import com.google.genai.types.Part

import java.util.UUID
import kotlinx.coroutines.runBlocking
import kotlinx.coroutines.flow.toList
import kotlinx.coroutines.reactive.asFlow

import io.reactivex.rxjava3.core.Flowable;

import io.modelcontextprotocol.client.transport.ServerParameters
import java.util.Optional
import kotlin.jvm.optionals.getOrNull
import java.util.concurrent.ConcurrentHashMap

# 🚀 Memory Management - Part 1 - Sessions

**Welcome to Day 3 of the Kaggle 5-day Agents course!**

        In this notebook, you'll learn:

- ✅ What sessions are and how to use them in your agent
        - ✅ How to build *stateful* agents with sessions and events
        - ✅ How to persist sessions in a database
        - ✅ Context management practices such as context compaction
        - ✅ Best practices for sharing session State

In [3]:
val apiKey = System.getenv("GOOGLE_API_KEY")
if (apiKey == null) {
    throw IllegalStateException("Please set the GOOGLE_API_KEY environment variable.")
}

In [4]:
val geminiClient = Client.builder().apiKey(apiKey).build()

In [5]:
val geminiModel = Gemini(
    "gemini-2.5-flash-lite", // Your model name
    geminiClient
)

---
## 🤹 Section 2: Session Management

### 2.1 The Problem

At their core, Large Language Models are **inherently stateless**. Their awareness is confined to the information you provide in a single API call. This means an agent without proper context management will react to the current prompt without considering any previous history.

**❓ Why does this matter?** Imagine trying to have a meaningful conversation with someone who forgets everything you've said after each sentence. That's the challenge we face with raw LLMs!

In ADK, we use `Sessions` for **short term memory management** and `Memory` for **long term memory.** In the next notebook, you'll focus on `Memory`.

### 2.2 What is a Session?

#### **📦 Session**

A session is a container for conversations. It encapsulates the conversation history in a chronological manner and also records all tool interactions and responses for a **single, continuous conversation**. A session is tied to a user and agent; it is not shared with other users. Similarly, a session history for an Agent is not shared with other Agents.

In ADK, a **Session** is comprised of two key components `Events` and `State`:

**📝 Session.Events**:

> While a session is a container for conversations, `Events` are the building blocks of a conversation.
>
> Example of Events:
> - *User Input*: A message from the user (text, audio, image, etc.)
> - *Agent Response*: The agent's reply to the user
> - *Tool Call*: The agent's decision to use an external tool or API
> - *Tool Output*: The data returned from a tool call, which the agent uses to continue its reasoning


**{} Session.State**:

> `session.state` is the Agent's scratchpad, where it stores and updates dynamic details needed during the conversation. Think of it as a global `{key, value}` pair storage which is available to all subagents and tools.

```
graph TD
    subgraph A["Agentic Application"];
        subgraph U["User"]
            subgraph S1["Session"]
                D1["Session.Events"]
                D2["Session.State"]
            end
        end
    end
```

<img src="https://storage.googleapis.com/github-repo/kaggle-5days-ai/day3/session-state-and-events.png" width="320" alt="Session state and events">

### 2.3 How to manage sessions?

An agentic application can have multiple users and each user may have multiple sessions with the application.
To manage these sessions and events, ADK offers a **Session Manager** and **Runner**.

1. **`SessionService`**: The storage layer
   - Manages creation, storage, and retrieval of session data
   - Different implementations for different needs (memory, database, cloud)

2. **`Runner`**: The orchestration layer
   - Manages the flow of information between user and agent
   - Automatically maintains conversation history
   - Handles the Context Engineering behind the scenes

Think of it like this:

- **Session** = A notebook 📓
- **Events** = Individual entries in a single page 📝
- **SessionService** = The filing cabinet storing notebooks 🗄️
- **Runner** = The assistant managing the conversation 🤖

In [6]:
val APP_NAME = "default"
val USER_ID = "default"
val SESSION_NAME = "default"
val MODEL_NAME = "gemini-2.5-flash-lite"

In [7]:
val rootAgent = LlmAgent.Builder()
    .name("text_chat_bot")
    .model(geminiModel)
    .description("A text chatbot")
    .build()

### 2.4 Implementing Our First Stateful Agent

        Let's build our first stateful agent, that can remember and have constructive conversations.

ADK offers different types of sessions suitable for different needs. As a start, we'll start with a simple Session Management option (`InMemorySessionService`):

In [8]:
val runner = InMemoryRunner(rootAgent, APP_NAME)
val sessionService = runner.sessionService()

In [9]:
println("Stateful agent initialized!")
println("   - Application: $APP_NAME")
println("   - User: $USER_ID")
println("   - Using ${sessionService.javaClass.name}")

Stateful agent initialized!
   - Application: default
   - User: default
   - Using com.google.adk.sessions.InMemorySessionService


### 1.4: Helper functions

        Helper function that manages a complete conversation session, handling session
        creation/retrieval, query processing, and response streaming. It supports
        both single queries and multiple queries in sequence.

Example:

```
>>> await run_session(runner, "What is the capital of France?", "geography-session")
>>> await run_session(runner, ["Hello!", "What's my name?"], "user-intro-session")
```

In [12]:
suspend fun runSession(
    runnerInstance: InMemoryRunner,
    userQueries: List<String> = emptyList(),
    sessionName: String = SESSION_NAME,
    appName: String = APP_NAME,
    userId: String = USER_ID,
    modelName: String = MODEL_NAME
) {
    println("\n ### Session: $sessionName")

    // Check if we can retrieve a session
    println("Getting session")
    var session = sessionService.getSession(
        appName, userId, sessionName, Optional.of(GetSessionConfig.builder().build())
    ).blockingGet()

    if (session == null) {
        println("Session does not exist, creating session")
        session = sessionService.createSession(
            appName, userId, ConcurrentHashMap<String?, Any?>(), sessionName
        ).blockingGet()
    }

    if (userQueries.isNotEmpty()) {
        for (queryText in userQueries) {
            println("\nUser > $queryText")

            // Convert the query string to the ADK Content format
            val queryContent = Content.builder()
                .role("user")
                .parts(Part.fromText(queryText))
                .build()

            // Stream the agent's response asynchronously
            // runAsync returns a Publisher, so we convert to Flow to collect events
            runnerInstance.runAsync(userId, session?.id(), queryContent)
                .asFlow()
                .collect { event ->

                    val part = event.content()
                        .orElse(null)
                        ?.parts()
                        ?.getOrNull()
                        ?.get(0)

                    val text = part?.text()

                    // Filter out empty or "None" responses
                    if (text != null && text.get() != "" && text.get() != "None") {
                        println("$modelName > ${text.get()}")
                    }
                }
        }
    } else {
        println("No queries!")
    }
}

suspend fun runSession(
    runnerInstance: InMemoryRunner,
    singleQuery: String,
    sessionName: String = "default"
) {
    runSession(runnerInstance, listOf(singleQuery), sessionName)
}

println("✅ Helper functions defined.")

✅ Helper functions defined.


### 2.5 Testing Our Stateful Agent

Now let's see the magic of sessions in action!

In [14]:
runBlocking {
    runSession(
        runner,
        listOf(
            "Hi, I am Santiago! What is the capital of the United States?",
            "Hello! What is my name?"
        ),
        "stateful-agentic-session"
    )
}


 ### Session: stateful-agentic-session
Getting session

User > Hi, I am Santiago! What is the capital of the United States?
gemini-2.5-flash-lite > Hello Santiago! The capital of the United States is Washington, D.C.

User > Hello! What is my name?
gemini-2.5-flash-lite > Hello! Your name is Santiago.


🎉 **Success!** The agent remembered your name because both queries were part of the same session. The Runner automatically maintained the conversation history.

But there's a catch: `InMemorySessionService` is temporary. **Once the application stops, all conversation history is lost.**


### 🛑 (Optional) 2.6 Testing Agent's forgetfulness

> To verify that the agent forgets the conversation, **restart the kernel**. Then, **run ALL the previous cells in this notebook EXCEPT the `run_session` in 2.5.**
>
> Now run the cell below. You'll see that the agent doesn't remember anything from the previous conversation.

In [13]:
// Run this cell after restarting the kernel. All this history will be gone...
runBlocking {
    runSession(
        runner,
        listOf(
            "What did I ask you about earlier?",
            "And remind me, what's my name?"
        ),
        "stateful-agentic-session"
    )
} // Note, we are using same session name


 ### Session: stateful-agentic-session
Getting session
Session does not exist, creating session

User > What did I ask you about earlier?
gemini-2.5-flash-lite > I do not have memory of past conversations. Therefore, I cannot recall what you asked me about earlier.

User > And remind me, what's my name?
gemini-2.5-flash-lite > I do not have access to your personal information, including your name. I am a large language model, trained by Google.


In [15]:
// Run this cell after running the code a couple cells ago, now it should remember.
runBlocking {
    runSession(
        runner,
        listOf(
            "What did I ask you about earlier?",
            "And remind me, what's my name?"
        ),
        "stateful-agentic-session"
    )
} // Note, we are using same session name


 ### Session: stateful-agentic-session
Getting session

User > What did I ask you about earlier?
gemini-2.5-flash-lite > You asked me about my capabilities and whether I remembered past conversations. I explained that I do not have memory of past conversations and cannot recall what you asked me about earlier.

User > And remind me, what's my name?
gemini-2.5-flash-lite > Your name is Santiago.
